# 前処理大全 SQL版 (Chapter3)

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
%load_ext sql
import os

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

# 演習問題

### 003_aggregation/01: カウントとユニークカウント

- `GROUP BY`で集約単位を指定し、`SELECT`に集約関数を指定する。
- `SELECT`内で利用している列で、`GROUP BY`で選択されていない列は、必ず集約関数が適用されている必要があります。
- 列名の前に`DISTINCT`があると、重複を削除できる。
  - `COUNT`のみである場合、NULLでないレコードすべてがカウントされる。

In [14]:
%%sql
SELECT
    hotel_id,
    COUNT(reserve_id) AS reserve_count,
    COUNT(DISTINCT customer_id) AS customer_count
FROM reserve_tb
GROUP BY hotel_id
ORDER BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,reserve_count,customer_count
h_1,10,10
h_10,3,3
h_100,20,19
h_101,17,17
h_102,13,13
h_103,10,10
h_104,11,11
h_105,15,15
h_106,9,9
h_107,11,11


### 003_aggregation/02: 合計値の算出

- 集約する場合、`AS`で名前を付けることがawesome。つけない場合勝手に付与され、わかりにくい名前となる。

In [20]:
%%sql
SELECT
    hotel_id,
    people_num,
    SUM(total_price) AS total_price_sum
FROM reserve_tb
GROUP BY hotel_id, people_num
ORDER BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,people_num,total_price_sum
h_1,3,391500
h_1,1,156600
h_1,4,417600
h_1,2,156600
h_10,1,11200
h_10,3,50400
h_10,4,67200
h_100,1,19200
h_100,3,158400
h_100,2,105600


### 003_aggregation/03: 極値、代表値の算出

- 平均値は、中央値やパーセンタイルよりも計算コストが低い。
- Postgresでは`MEDIAN`が使えず、代わりに`PERCENTILECONT(0.5)`を使う。
- `WITHIN GROUP`は覚える必要がある。
- `WITHIN GROUP`内の`ORDER BY`を付ける必要があり、PERCENTILEを算出する場合の並べ方が指定できる
  - 20パーセンタイルを逆順(`DESC`)にすると、80パーセンタイルとなる。
- Redshiftの場合、`APPROXIMATE PERCENTILE_DISC`による近似計算により高速に計算ができる。
  - https://docs.aws.amazon.com/ja_jp/redshift/latest/dg/r_APPROXIMATE_PERCENTILE_DISC.html

In [36]:
%%sql
SELECT
    hotel_id,
    MAX(total_price) AS price_max,
    MIN(total_price) AS price_min,
    AVG(total_price) AS price_ave,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY total_price) AS price_med,
    PERCENTILE_CONT(0.2) WITHIN GROUP(ORDER BY total_price) AS price_per20
FROM reserve_tb
GROUP BY hotel_id
ORDER BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,price_max,price_min,price_ave,price_med,price_per20
h_1,208800,26100,112230.000000000000,104400.0,73080.0
h_10,67200,11200,42933.333333333333,50400.0,26880.0
h_100,57600,4800,27600.000000000000,28800.0,9600.0
h_101,168000,14000,75764.705882352941,56000.0,30800.000000000004
h_102,72000,12000,32769.230769230769,24000.0,18000.0
h_103,194400,32400,93960.000000000000,64800.0,48600.0
h_104,379800,42200,176472.727272727273,126600.0,84400.0
h_105,82400,20600,48753.333333333333,41200.0,30900.0
h_106,199800,44400,101133.333333333333,88800.0,57720.0
h_107,225600,37600,107672.727272727273,112800.0,75200.0


### 003_aggregation/04: ばらつき具合の算出

- `VARIANCE`と`STDDEV`は、データ数が1件の場合、NULLを返す。
- `COALESCE`はNULLの場合に値を埋めることができる。
  - 実際は引数の順番に優先される処理となります。
  - (下記の場合は２番目に固定値なので必ずそこで終わる)

In [43]:
%%sql
SELECT
    hotel_id,
    COALESCE(VARIANCE(total_price), 0) AS price_var,
    COALESCE(STDDEV(total_price), 0) AS price_std
FROM reserve_tb
GROUP BY hotel_id
ORDER BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,price_var,price_std
h_1,3186549000.00000000,56449.52612733
h_10,825813333.33333333,28736.96806090
h_100,319831578.94736842,17883.83568889
h_101,2402441176.47058824,49014.70367625
h_102,357692307.69230769,18912.75515868
h_103,3545856000.00000000,59547.09060903
h_104,11688786181.81818182,108114.68994461
h_105,416276952.38095238,20402.86627856
h_106,3217150000.00000000,56719.92595200
h_107,2833946181.81818182,53234.82114010


### 003_aggregation/05: 最頻値の算出

#### Not awesomeな例: WITH句を用いた例

- `ROUND`で四捨五入し、その四捨五入をクラスとして`GROUP BY`して件数カウントする。
  - 第二引数は小数点位置を表す。マイナスで1000単位などに丸められる。
- その後`WHERE`と`MAX`で最大件数のみを抽出する。
  - `(SELECT MAX(reserve_count) FROM reserve_count_table)`の部分をサブクエリと呼ぶ。
  - 以前はサブクエリはアンチパターンであったが、現在はシンプルなサブクエリであれば問題ないとされている。
- しかし、`WITH`句等もあり複雑なため、Not awesomeなパターン。

In [58]:
%%sql
WITH reserve_count_table AS (
    SELECT
        ROUND(total_price, -3) AS total_price_round,
        COUNT(*) AS reserve_count
    FROM reserve_tb
    GROUP BY total_price_round
)
SELECT
    total_price_round
FROM reserve_count_table
WHERE reserve_count = (SELECT MAX(reserve_count) FROM reserve_count_table)

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


total_price_round
20000


### awesomeな例: ORDER BYを使用

- `MAX`, `WHERE`の代わりに`ORDER BY`を使う。
- `ORDER BY`のなかに`COUNT`など集約関数を使用できる。

In [66]:
%%sql
SELECT
    ROUND(total_price, -3) AS total_price_round
FROM reserve_tb
GROUP BY total_price_round
ORDER BY COUNT(*) DESC
LIMIT 1

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


total_price_round
20000


### 003_aggregation/06: 順位の算出

- 順位付けは計算コストがかかる（並べ替えが必要なため）
- そのためグループ毎など小分けにして行うことで工夫ができる。
- それに使用するのがWINDOW関数などである。
- SQLの場合順位付けは、関数の種類により同値をどのように扱うか制御する。
  - `RANK` ... 同値は小さい値（高順位）で同じ順位をつける。
  - `ROW_NUMBER` ... 元の並び順につける。
- `RANK`や`ROW_NUMBER`がWINDOW関数であり、後続に`OVER`句もつける。

#### customer_id毎に日付順の順位を付与する例

In [10]:
%%sql
SELECT
    *,
    RANK()
        OVER (PARTITION BY customer_id ORDER BY people_num) AS log_no
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,1
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,1
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,3
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,5
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,7
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,7
r61,h_62,c_10,2017-03-12 00:45:50,2017-04-08,10:00:00,2017-04-10,2,187200,1
r63,h_142,c_10,2017-09-11 19:41:28,2017-09-13,10:00:00,2017-09-15,2,37200,1


#### PARTITIN BYをなくした場合

- `PARTITON BY`がない場合、customer_idを考慮せず全体で順位付けがなされる。

In [17]:
%%sql
SELECT
    *,
    ROW_NUMBER()
        OVER (ORDER BY reserve_datetime) AS log_no
FROM reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
r2757,h_169,c_696,2016-01-01 08:56:43,2016-01-15,11:30:00,2016-01-18,4,100800,1
r1974,h_156,c_493,2016-01-01 09:40:10,2016-01-26,09:00:00,2016-01-28,1,36800,2
r2646,h_142,c_665,2016-01-01 11:07:09,2016-01-02,09:00:00,2016-01-05,4,111600,3
r2444,h_129,c_611,2016-01-01 11:25:41,2016-01-26,12:30:00,2016-01-27,2,16000,4
r3286,h_241,c_826,2016-01-01 12:20:12,2016-01-02,09:30:00,2016-01-04,2,24000,5
r1920,h_299,c_475,2016-01-01 19:35:25,2016-01-13,12:30:00,2016-01-16,3,217800,6
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,7
r927,h_109,c_227,2016-01-02 02:54:18,2016-01-10,10:30:00,2016-01-12,4,84000,8
r479,h_179,c_110,2016-01-02 05:24:11,2016-01-21,10:30:00,2016-01-24,3,50400,9
r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,10


#### ORDER BYをなくした場合

- ただの登場順になるっぽい。

In [45]:
%%sql
SELECT
    *,
    ROW_NUMBER()
        OVER () AS log_no
FROM reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,6
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,7
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,8
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,9
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,10


- RANKの場合、すべて同じ順位になる。

In [47]:
%%sql
SELECT
    *,
    RANK()
        OVER () AS log_no
FROM reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,1
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,1
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,1
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,1
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,1
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,1
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,1
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,1
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,1


#### 集約とWINDOWを同時に用いる場合

- 集約関数と同時に用いる場合。
- GROUP BYによる集約結果の順位付けをすることが可能。
  - この場合、ORDER BYに集約関数を書く。

In [79]:
%%sql
SELECT
    hotel_id,
    COUNT(*),
    RANK()
        OVER (ORDER BY COUNT(*) DESC) AS reserve_count_rank
FROM reserve_tb
GROUP BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,count,reserve_count_rank
h_241,27,1
h_144,25,2
h_37,24,3
h_142,24,3
h_178,23,5
h_82,23,5
h_215,22,7
h_34,22,7
h_204,22,7
h_56,21,10


#### その他の順序関数: DENSE_RANK

In [80]:
%%sql
SELECT
    hotel_id,
    COUNT(*),
    DENSE_RANK()
        OVER (ORDER BY COUNT(*) DESC) AS reserve_count_rank
FROM reserve_tb
GROUP BY hotel_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


hotel_id,count,reserve_count_rank
h_241,27,1
h_144,25,2
h_37,24,3
h_142,24,3
h_178,23,4
h_82,23,4
h_215,22,5
h_34,22,5
h_204,22,5
h_56,21,6


### 補足:いろいろなWINDOW関数の使い方

#### SUM(?) OVER()

- MAX,MIN,AVEなどは同じ感じの仕様。
- `OVER()`とすれば全体の合計がすべてのレコードに格納される。

In [48]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER()
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,10245
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,10245
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,10245
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,10245
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,10245
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,10245
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,10245
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,10245
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,10245
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,10245


#### SUM(?) OVER (PARTITION BY ?)
- `PARTITION BY`を書くと、そのPARTITION毎の合計値が格納される。

In [49]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(PARTITION BY hotel_id)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r330,h_1,c_76,2016-12-25 12:02:22,2016-12-30,10:00:00,2017-01-01,4,208800,26
r1762,h_1,c_437,2016-06-20 15:26:53,2016-07-09,09:00:00,2016-07-11,1,52200,26
r2155,h_1,c_535,2016-12-02 21:56:33,2016-12-08,12:30:00,2016-12-11,1,78300,26
r3386,h_1,c_845,2016-02-13 18:48:17,2016-03-01,11:00:00,2016-03-03,3,156600,26
r1901,h_1,c_469,2016-10-28 06:16:14,2016-11-21,10:30:00,2016-11-22,1,26100,26
r2250,h_1,c_561,2018-04-14 10:23:17,2018-04-30,09:30:00,2018-05-01,4,104400,26
r3222,h_1,c_808,2017-04-12 14:07:48,2017-04-19,11:30:00,2017-04-21,3,156600,26
r2533,h_1,c_632,2017-02-07 21:36:39,2017-02-08,12:30:00,2017-02-09,4,104400,26
r210,h_1,c_49,2016-07-09 23:28:18,2016-08-05,12:00:00,2016-08-08,2,156600,26
r1510,h_1,c_371,2016-03-11 17:44:52,2016-03-19,11:30:00,2016-03-20,3,78300,26


#### SUM(?) OVER (ORDER BY ?)

- `ORDER BY`を使うと累積和となる。

In [67]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,8
r100,h_64,c_17,2017-11-21 21:53:25,2017-12-08,12:30:00,2017-12-09,3,48000,11
r1000,h_159,c_244,2017-12-18 22:30:54,2018-01-16,11:00:00,2018-01-19,3,582300,14
r1001,h_6,c_244,2018-05-04 01:46:32,2018-05-11,10:30:00,2018-05-14,2,297000,16
r1002,h_290,c_244,2018-09-26 07:41:44,2018-10-20,10:00:00,2018-10-23,4,190800,20
r1003,h_18,c_245,2016-06-08 14:47:38,2016-07-08,11:00:00,2016-07-11,4,333600,24
r1004,h_214,c_245,2016-07-31 12:32:40,2016-08-13,12:00:00,2016-08-16,3,437400,27
r1005,h_176,c_246,2016-02-29 19:45:59,2016-03-23,11:30:00,2016-03-25,3,104400,30
r1006,h_172,c_247,2016-03-14 17:08:50,2016-03-28,10:00:00,2016-03-30,4,128000,34


#### SUM(?) OVER (ORDER BY ?) + FRAME句

- `ORDER BY`を使う場合、`FRAME`句を使える。以下のように現レコードに対する。WINDOW範囲を指定できる。
  - 以下はPostgresの未指定と同じ設定。Redshiftでは未指定ができない（必ず指定が必要）。
    - `UNBOUNDED PRECEDING`: PRECEDING(先行側)は境界無し。
    - `0 FOLLOWING`: 後述側は現レコードまで。
  - 注意点として、ランキング系のWINDOW関数では、FRAME句を指定できない。
    - 性質が違うから当然ではある。

In [54]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id ROWS BETWEEN UNBOUNDED PRECEDING AND 0 FOLLOWING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,8
r100,h_64,c_17,2017-11-21 21:53:25,2017-12-08,12:30:00,2017-12-09,3,48000,11
r1000,h_159,c_244,2017-12-18 22:30:54,2018-01-16,11:00:00,2018-01-19,3,582300,14
r1001,h_6,c_244,2018-05-04 01:46:32,2018-05-11,10:30:00,2018-05-14,2,297000,16
r1002,h_290,c_244,2018-09-26 07:41:44,2018-10-20,10:00:00,2018-10-23,4,190800,20
r1003,h_18,c_245,2016-06-08 14:47:38,2016-07-08,11:00:00,2016-07-11,4,333600,24
r1004,h_214,c_245,2016-07-31 12:32:40,2016-08-13,12:00:00,2016-08-16,3,437400,27
r1005,h_176,c_246,2016-02-29 19:45:59,2016-03-23,11:30:00,2016-03-25,3,104400,30
r1006,h_172,c_247,2016-03-14 17:08:50,2016-03-28,10:00:00,2016-03-30,4,128000,34


- なおフレーム数について負の値は指定できない。

In [55]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id ROWS BETWEEN UNBOUNDED PRECEDING AND -1 FOLLOWING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db


DataError: (psycopg2.errors.InvalidPrecedingOrFollowingSize) フレームの終了オフセットは負数であってはなりません

[SQL: SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id ROWS BETWEEN UNBOUNDED PRECEDING AND -1 FOLLOWING)
FROM reserve_tb
LIMIT 20]
(Background on this error at: https://sqlalche.me/e/14/9h9h)

- 負を使う代わりに両方`PRECEDING`にすればOK

In [58]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,None
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,4
r100,h_64,c_17,2017-11-21 21:53:25,2017-12-08,12:30:00,2017-12-09,3,48000,8
r1000,h_159,c_244,2017-12-18 22:30:54,2018-01-16,11:00:00,2018-01-19,3,582300,11
r1001,h_6,c_244,2018-05-04 01:46:32,2018-05-11,10:30:00,2018-05-14,2,297000,14
r1002,h_290,c_244,2018-09-26 07:41:44,2018-10-20,10:00:00,2018-10-23,4,190800,16
r1003,h_18,c_245,2016-06-08 14:47:38,2016-07-08,11:00:00,2016-07-11,4,333600,20
r1004,h_214,c_245,2016-07-31 12:32:40,2016-08-13,12:00:00,2016-08-16,3,437400,24
r1005,h_176,c_246,2016-02-29 19:45:59,2016-03-23,11:30:00,2016-03-25,3,104400,27
r1006,h_172,c_247,2016-03-14 17:08:50,2016-03-28,10:00:00,2016-03-30,4,128000,30


- プラスマイナス1の範囲で合計３つのSUMとする場合

In [63]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(ORDER BY reserve_id ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,8
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,11
r100,h_64,c_17,2017-11-21 21:53:25,2017-12-08,12:30:00,2017-12-09,3,48000,10
r1000,h_159,c_244,2017-12-18 22:30:54,2018-01-16,11:00:00,2018-01-19,3,582300,8
r1001,h_6,c_244,2018-05-04 01:46:32,2018-05-11,10:30:00,2018-05-14,2,297000,9
r1002,h_290,c_244,2018-09-26 07:41:44,2018-10-20,10:00:00,2018-10-23,4,190800,10
r1003,h_18,c_245,2016-06-08 14:47:38,2016-07-08,11:00:00,2016-07-11,4,333600,11
r1004,h_214,c_245,2016-07-31 12:32:40,2016-08-13,12:00:00,2016-08-16,3,437400,10
r1005,h_176,c_246,2016-02-29 19:45:59,2016-03-23,11:30:00,2016-03-25,3,104400,10
r1006,h_172,c_247,2016-03-14 17:08:50,2016-03-28,10:00:00,2016-03-30,4,128000,9


#### SUM(?) OVER (PARTITION BY ? ORDER BY ?) + FRAME句

- `PARTITION BY`を付けることももちろん可能。

In [57]:
%%sql
SELECT
    *,
    SUM(people_num)
        OVER(PARTITION BY hotel_id ORDER BY reserve_id ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,sum
r1510,h_1,c_371,2016-03-11 17:44:52,2016-03-19,11:30:00,2016-03-20,3,78300,4
r1762,h_1,c_437,2016-06-20 15:26:53,2016-07-09,09:00:00,2016-07-11,1,52200,5
r1901,h_1,c_469,2016-10-28 06:16:14,2016-11-21,10:30:00,2016-11-22,1,26100,4
r210,h_1,c_49,2016-07-09 23:28:18,2016-08-05,12:00:00,2016-08-08,2,156600,4
r2155,h_1,c_535,2016-12-02 21:56:33,2016-12-08,12:30:00,2016-12-11,1,78300,7
r2250,h_1,c_561,2018-04-14 10:23:17,2018-04-30,09:30:00,2018-05-01,4,104400,9
r2533,h_1,c_632,2017-02-07 21:36:39,2017-02-08,12:30:00,2017-02-09,4,104400,11
r3222,h_1,c_808,2017-04-12 14:07:48,2017-04-19,11:30:00,2017-04-21,3,156600,11
r330,h_1,c_76,2016-12-25 12:02:22,2016-12-30,10:00:00,2017-01-01,4,208800,10
r3386,h_1,c_845,2016-02-13 18:48:17,2016-03-01,11:00:00,2016-03-03,3,156600,7


#### FIRST_VALUE

In [66]:
%%sql
SELECT
    *,
    FIRST_VALUE(people_num)
        OVER()
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,first_value
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,4
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,4
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,4
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,4
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,4
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,4
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,4
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,4


- FRAME句などもSUMと同じや

In [74]:
%%sql
SELECT
    *,
    FIRST_VALUE(people_num)
        OVER(PARTITION BY hotel_id ORDER BY reserve_id ROWS BETWEEN 1 FOLLOWING AND 2 FOLLOWING)
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,first_value
r1510,h_1,c_371,2016-03-11 17:44:52,2016-03-19,11:30:00,2016-03-20,3,78300,1
r1762,h_1,c_437,2016-06-20 15:26:53,2016-07-09,09:00:00,2016-07-11,1,52200,1
r1901,h_1,c_469,2016-10-28 06:16:14,2016-11-21,10:30:00,2016-11-22,1,26100,2
r210,h_1,c_49,2016-07-09 23:28:18,2016-08-05,12:00:00,2016-08-08,2,156600,1
r2155,h_1,c_535,2016-12-02 21:56:33,2016-12-08,12:30:00,2016-12-11,1,78300,4
r2250,h_1,c_561,2018-04-14 10:23:17,2018-04-30,09:30:00,2018-05-01,4,104400,4
r2533,h_1,c_632,2017-02-07 21:36:39,2017-02-08,12:30:00,2017-02-09,4,104400,3
r3222,h_1,c_808,2017-04-12 14:07:48,2017-04-19,11:30:00,2017-04-21,3,156600,4
r330,h_1,c_76,2016-12-25 12:02:22,2016-12-30,10:00:00,2017-01-01,4,208800,3
r3386,h_1,c_845,2016-02-13 18:48:17,2016-03-01,11:00:00,2016-03-03,3,156600,None


#### LAG/LEAD

- LAGで過去の、LEADで未来の値を取得できる。
- FRAME句を書けるが意味についてはどういう動きになるのかあまりわかってない。

In [94]:
%%sql
SELECT
    *,
    LAG(total_price, 2)
    OVER(ORDER BY reserve_datetime) AS before_price
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
r2757,h_169,c_696,2016-01-01 08:56:43,2016-01-15,11:30:00,2016-01-18,4,100800,None
r1974,h_156,c_493,2016-01-01 09:40:10,2016-01-26,09:00:00,2016-01-28,1,36800,None
r2646,h_142,c_665,2016-01-01 11:07:09,2016-01-02,09:00:00,2016-01-05,4,111600,100800
r2444,h_129,c_611,2016-01-01 11:25:41,2016-01-26,12:30:00,2016-01-27,2,16000,36800
r3286,h_241,c_826,2016-01-01 12:20:12,2016-01-02,09:30:00,2016-01-04,2,24000,111600
r1920,h_299,c_475,2016-01-01 19:35:25,2016-01-13,12:30:00,2016-01-16,3,217800,16000
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,24000
r927,h_109,c_227,2016-01-02 02:54:18,2016-01-10,10:30:00,2016-01-12,4,84000,217800
r479,h_179,c_110,2016-01-02 05:24:11,2016-01-21,10:30:00,2016-01-24,3,50400,25200
r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,84000


- ということは、LAGはFIRST_VALUEとかで代用できるな。

In [95]:
%%sql
SELECT
    *,
    FIRST_VALUE(total_price)
    OVER(ORDER BY reserve_datetime ROWS BETWEEN 2 PRECEDING AND 2 PRECEDING) AS before_price
FROM reserve_tb
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
r2757,h_169,c_696,2016-01-01 08:56:43,2016-01-15,11:30:00,2016-01-18,4,100800,None
r1974,h_156,c_493,2016-01-01 09:40:10,2016-01-26,09:00:00,2016-01-28,1,36800,None
r2646,h_142,c_665,2016-01-01 11:07:09,2016-01-02,09:00:00,2016-01-05,4,111600,100800
r2444,h_129,c_611,2016-01-01 11:25:41,2016-01-26,12:30:00,2016-01-27,2,16000,36800
r3286,h_241,c_826,2016-01-01 12:20:12,2016-01-02,09:30:00,2016-01-04,2,24000,111600
r1920,h_299,c_475,2016-01-01 19:35:25,2016-01-13,12:30:00,2016-01-16,3,217800,16000
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,24000
r927,h_109,c_227,2016-01-02 02:54:18,2016-01-10,10:30:00,2016-01-12,4,84000,217800
r479,h_179,c_110,2016-01-02 05:24:11,2016-01-21,10:30:00,2016-01-24,3,50400,25200
r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,84000
